In [ ]:
%%bash

# Dependencies
pip install docx2txt
pip install gensim
pip install keras
pip install nltk
pip install -U scikit-learn
pip install python-docx
pip install tensorflow

if ls docx2csv >/dev/null 2>&1; then
    echo "docx2csv exists."
else
    echo "Folder does not exist. Cloning docx2csv."
    git clone https://github.com/ivbeg/docx2csv.git
fi

In [ ]:
# ----- TEST DATA INPUT -----

# Computer Science Test Data.
# CURRENT_MAPPING="Lists_ComputerScience.docx"
# ORIGINAL_MAPPING="Original-Mapping-ComputerScience.csv"

# InformationSecurity Test Data.
CURRENT_MAPPING="Lists_InformationSecurity.docx"
ORIGINAL_MAPPING="Original-Mapping-InfoSecurity.csv"

In [ ]:
from gensim.models import Word2Vec
from gensim.test.utils import common_texts

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()


In [ ]:
# extract tables from word document
from docx2csv import extract_tables, extract
tables = extract_tables(CURRENT_MAPPING)


In [ ]:
from docx import Document
document = Document(CURRENT_MAPPING)

In [ ]:
# Array of all the PLOs and ULOs (We can couple them together as we're trying to identify Bloom/Solo level here)
lo_sentence_array = []

# TODO: train CLO classification with all data instead of just one course.
for table in document.tables:
    for row in table.rows:
        for cell in row.cells[1:]:
            tokens = nltk.word_tokenize(cell.text)
            cleaned_tokens = [lemmatizer.lemmatize(token) for token in tokens if token.lower() not in stop_words]
            lo_sentence_array.append(cleaned_tokens)

# build the vocabulary and train the model
# IMPORTANT, N0TE THAT sg=1 flag specifies Word2Vec to use the Skip Gram Model as designated by the LSTM paper.
model = Word2Vec(sentences=lo_sentence_array,vector_size=100, window=5, min_count=1, workers=4, sg=1)

# train the model with the course's ULOs and PLOs.
model.train([tokens], total_examples=len([tokens]), epochs=10)

lo_sentence_array

In [ ]:
import docx2txt
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import RMSprop

# load in the Word document using docx2txt
doc_text = docx2txt.process(CURRENT_MAPPING)

# preprocess the text using NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# tokenize the text and remove stop words and non-alphabetic characters
tokens = [word.lower() for word in word_tokenize(doc_text) if word.isalpha() and word.lower() not in stop_words]

# lemmatization
lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]


# convert tokens back to single string format
corpus = ' '.join(lemmatized_tokens)

# create a tokenizer and fit on the corpus
tokenizer = Tokenizer()
tokenizer.fit_on_texts([corpus])

# convert the text to a sequence of integers
# sequences = tokenizer.texts_to_sequences([corpus])

# pad the sequences to have a fixed length
max_length = 50
# padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

# LSTM model
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 50

model = Sequential([
    tf.keras.layers.Embedding(10000, 16),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.2), # Dropout rate set to 0.2 as specified from the paper
    tf.keras.layers.Dense(6, activation='softmax')
])

model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
model.add(LSTM(units=64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(units=3, activation='softmax'))

# RMS Optimizer as specified by the paper.
optimizer = RMSprop(learning_rate=0.001)

# compile the model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# train the model
# X_train and y_train are assumed to be already defined
# 
# model.fit(X_train, y_train, epochs=10, batch_size=32)

In [ ]:
import io
import re
import string
import tqdm

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

## training data LO
## sentence = "use big data streaming technologies."
## word = "apply"
## categories = ["Remembering", "Understanding", "Applying", "Analysing", "Evaluating", "Creating"]



# Load the TensorBoard notebook extension
%load_ext tensorboard

SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

sentence = "The wide road shimmered in the hot sun"
tokens = list(sentence.lower().split())
print(len(tokens))

vocab, index = {}, 1  # start indexing from 1
vocab['<pad>'] = 0  # add a padding token
for token in tokens:
  if token not in vocab:
    vocab[token] = index
    index += 1
vocab_size = len(vocab)
print(vocab)

inverse_vocab = {index: token for token, index in vocab.items()}
print(inverse_vocab)

example_sequence = [vocab[word] for word in tokens]
print(example_sequence)

window_size = 2
positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
      example_sequence,
      vocabulary_size=vocab_size,
      window_size=window_size,
      negative_samples=0)
print(len(positive_skip_grams))


In [ ]:
## packages to install
%%bash

python3 -m spacy download en_core_web_sm

In [36]:
## This code loads the vector file into the word_vectors variable
## Note: This piece of code may take upto an hour or two to run depending on your pc specs.
## My i5 8th gen with 8gig ram took 58mins to run.

from gensim.models import KeyedVectors

# Path to the downloaded .vec file
path_to_vectors = 'wiki-news-300d-1M.vec'
# Load the word vectors
word_vectors = KeyedVectors.load_word2vec_format(path_to_vectors)

# Find similar words
similar_words = word_vectors.most_similar('cat')

# Calculate word similarity
similarity = word_vectors.similarity('cat', 'dog')

# Perform vector arithmetic
result = word_vectors['king'] - word_vectors['man'] + word_vectors['woman']




test1
test2


In [43]:
import io
import re
import string
import tqdm

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

## training data LO
## sentence = "use big data streaming technologies."
## word = "apply"
## categories = ["Remembering", "Understanding", "Applying", "Analysing", "Evaluating", "Creating"]



# Load the TensorBoard notebook extension
%load_ext tensorboard

SEED = 42
AUTOTUNE = tf.data.AUTOTUNE


def generate_skipgrams(sentence):
    tokens = list(sentence.lower().split())
    print(len(tokens))

    vocab, index = {}, 1  # start indexing from 1
    vocab['<pad>'] = 0  # add a padding token
    for token in tokens:
      if token not in vocab:
        vocab[token] = index
        index += 1
    vocab_size = len(vocab)
    print(vocab)

    inverse_vocab = {index: token for token, index in vocab.items()}
    print(inverse_vocab)

    example_sequence = [vocab[word] for word in tokens]
    print(example_sequence)

    window_size = 2
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          example_sequence,
          vocabulary_size=vocab_size,
          window_size=window_size,
          negative_samples=0)
    
    return positive_skip_grams

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [66]:
import spacy

## Function to identify verbs in a sentence
def identify_verbs(sentence):
    # Load the English language model in spaCy
    nlp = spacy.load('en_core_web_sm')
    
    # Process the sentence using spaCy
    doc = nlp(sentence)
    
    # Extract the verbs from the processed sentence
    verbs = [token.lemma_ for token in doc if token.pos_ == 'VERB']
    
    return verbs

In [84]:
## Main piece of code that performs the mapping 

sentences = [
    "apply common data analytics and machine learning algorithms in a big data environment.",
    "use big data streaming technologies."
]
bloom_levels = ["Remembering", "Understanding", "Applying", "Analysing", "Evaluating", "Creating"]

# identified_levels = []
final_level = None

for i in range(len(sentences)):
    verbs = identify_verbs(sentences[i])
    print(verbs)
    score = 0
    for j in range(len(verbs)):
        for k in range(len(bloom_levels)):
            similarity_score = word_vectors.similarity(verbs[j], bloom_levels[k])
            if similarity_score >= score:
                score=similarity_score
                final_level = bloom_levels[k]
    print("Sentence: ", sentences[i], " Identified blooms level: ", final_level)


### Todos
# Find a way to use skipgrams
# This method only works for blooms since this paper is only based on blooms mapping


['apply', 'learn']
Sentence:  apply common data analytics and machine learning algorithms in a big data environment.  Identified blooms level:  Applying
['use']
Sentence:  use big data streaming technologies.  Identified blooms level:  Applying
